*fuzzy programming*
*Generating the sets


In [5]:
import numpy as np
from scipy.optimize import linprog
#from random import random
import random


random.seed(10)
# Problem parameters, you can change the required problem size here
PROBLEM_SIZE = 3000
NUM_SETS =3000
SETS = tuple(
    np.array([random.random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

Let's build A* as a basis for comparison

In [6]:
from random import random
from functools import reduce
import numpy as np
from queue import PriorityQueue
from collections import namedtuple

In [7]:
start = time.time()
State = namedtuple('State', ['taken', 'not_taken'])


def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )


def g_function(state):
    return len(state.taken)


def h_function(state):
    return distance(state)


def costFunction(state):
    return g_function(state) + h_function(state)


assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

frontier = PriorityQueue()
start_state = State(set(), set(range(NUM_SETS)))
frontier.put((costFunction(start_state), start_state))
visited_states = set()

counter = 0
while not frontier.empty():
    counter += 1
    _, current_state = frontier.get()

    # Convert the current_state to a frozenset before adding it to visited_states
    current_state_frozen = (frozenset(current_state.taken), frozenset(current_state.not_taken))

    if current_state_frozen not in visited_states:
        visited_states.add(current_state_frozen)

        if goal_check(current_state):
            break

        for action in current_state.not_taken:
            new_state = State(
                current_state.taken | {action},
                current_state.not_taken - {action},
            )

            # Convert the new_state to a frozenset before checking for membership
            new_state_frozen = (frozenset(new_state.taken), frozenset(new_state.not_taken))
            if new_state_frozen not in visited_states:
                frontier.put((costFunction(new_state), new_state))

print(f"Solved in {counter:,} steps ({len(current_state.taken)} sets selected)")
print("Selected sets:", current_state.taken)

end = time.time()
#print(f"Solved in {counter:,} steps ({len(current_state.taken)} sets selected)")
#print("Selected sets:", current_state.taken)
print("duration for A_star method is:  ", end-start)

Solved in 13 steps (12 sets selected)
Selected sets: {1505, 1825, 121, 644, 645, 241, 883, 2934, 2520, 217, 2076, 254}
duration for A_star method is:   38.740283489227295


Fuzzy heuristics utilized in the report


In [8]:
import time
start  = time.time()
# Define the elements and their fuzzy membership functions
elements = [f"Element{i+1}" for i in range(PROBLEM_SIZE)]
membership_values = np.random.rand(2, PROBLEM_SIZE)

# Create a dictionary to store the fuzzy membership values for each element
fuzzy_values_dict = {elements[i]: membership_values[:, i] for i in range(PROBLEM_SIZE)}

# Perform fuzzy heuristics to select sets
selected_sets = []
covered_elements = set()
while len(covered_elements) < PROBLEM_SIZE:
    best_set = None
    best_score = -1
    for i, s in enumerate(SETS):
        score = sum(max(fuzzy_values_dict[elem]) for elem, included in zip(elements, s) if elem not in covered_elements and included)
        if score > best_score:
            best_score = score
            best_set = i
    if best_set is not None:
        selected_sets.append(best_set)
        covered_elements.update([elements[j] for j in range(PROBLEM_SIZE) if SETS[best_set][j]])
    else:
        break

# Print the results
end = time.time()
print("this is the duration of fuzzy heuristics: ", end - start)
print("Selected sets:", selected_sets)

this is the duration of fuzzy heuristics:  19.705235958099365
Selected sets: [2934, 241, 2076, 230, 66, 1875, 1791, 1452, 104, 1972, 1311, 1082]


In [ ]:
fuzzy_values_dict

{'Element1': array([0.01122153, 0.78734784]),
 'Element2': array([0.93933409, 0.78005747]),
 'Element3': array([0.92564007, 0.99585693]),
 'Element4': array([0.79272218, 0.13313239]),
 'Element5': array([0.08707163, 0.68446879]),
 'Element6': array([0.97517249, 0.45214105]),
 'Element7': array([0.7859037 , 0.57484935]),
 'Element8': array([0.82231762, 0.86939653]),
 'Element9': array([0.49274516, 0.02320722]),
 'Element10': array([0.57663198, 0.80708316]),
 'Element11': array([0.12302898, 0.08154713]),
 'Element12': array([0.09174296, 0.51328931]),
 'Element13': array([0.39069728, 0.16011154]),
 'Element14': array([0.07114193, 0.74606873]),
 'Element15': array([0.38872296, 0.4641369 ]),
 'Element16': array([0.39057673, 0.1968132 ]),
 'Element17': array([0.30476062, 0.9097754 ]),
 'Element18': array([0.1806829 , 0.15019688]),
 'Element19': array([0.85078607, 0.55817705]),
 'Element20': array([0.46325633, 0.08866859]),
 'Element21': array([0.1697781, 0.6867266]),
 'Element22': array([0.8

Neglect the following


In [ ]:
import numpy as np
from scipy.optimize import linprog
import random

random.seed(10)

# Problem parameters
PROBLEM_SIZE = 1000
NUM_SETS = 1000
SETS = tuple(
    np.array([random.random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

# Set up the set covering problem
c = np.ones(NUM_SETS)  # Coefficients of the objective function
A = np.vstack(SETS)    # Coefficients of the constraints
b = np.ones(PROBLEM_SIZE)  # Right-hand side of the constraints

# Solve the linear program
res = linprog(c, A_eq=A, b_eq=b, method='highs')

# Check if the problem is feasible
if res.success:
    print("Solution found!")
    print("Selected sets (indices):", [i for i, x in enumerate(res.x) if x > 0.5])
else:
    print("No solution found.")

In [ ]:
import time
start = time.time()
# Define the elements and their fuzzy membership functions
elements = [f"Element{i+1}" for i in range(PROBLEM_SIZE)]

# Generate random fuzzy membership values for the elements
membership_values = np.random.rand(2, PROBLEM_SIZE)

# Define the constraints and the cost function for the set covering problem
A = np.array(SETS, dtype=int)
b = np.ones(NUM_SETS)
c = np.random.rand(PROBLEM_SIZE)

# Convert the fuzzy membership values to crisp values
crisp_values = np.mean(membership_values, axis=0)  # Use the mean as a representative crisp value

# Define the linear programming problem
res = linprog(c, A_ub=-A, b_ub=-b, bounds=(0, 1))

# Print the results
if res.success:
    print("Set covering solution:")
    # for i, el in enumerate(elements):
    #     print(f"{el}: {res.x[i]}")
    print(f"Cost: {res.fun}")
    selected_sets = [i for i, x in enumerate(res.x) if x > 0.1]
    print("Selected sets:", selected_sets)
else:
    print("No feasible solution found.")
end = time.time()
print("duration for fuzzy method is:  ", end-start)

In [ ]:
s=SETS[354] | SETS[4824] | SETS[1079] | SETS[1094]| SETS[2657]| SETS[4830]| SETS[4324]| SETS[1973]| SETS[2729]| SETS[2119]| SETS[2555]| SETS[3346]| SETS[976]
print(sum(s))

5000


In [ ]:
#s=SETS[68] | SETS[95]
s=SETS[3] | SETS[10] | SETS[37] | SETS[48]| SETS[67]| SETS[84]| SETS[85]| SETS[99] #| SETS[319]| SETS[339]| SETS[358]| SETS[377]| SETS[430]| SETS[446]| SETS[469]| SETS[592]| SETS[664]| SETS[669]| SETS[705]| SETS[800]| SETS[899]| SETS[925]| SETS[998]
#print(s)
print(sum(s))
3, 10, 37, 48, 67, 84, 85, 99

*fuzzy meta heuristics with fixed population*

In [ ]:
from random import random, sample
start = time.time()
# Define the elements and their fuzzy membership functions
elements = [f"Element{i+1}" for i in range(PROBLEM_SIZE)]
membership_values = np.random.rand(2, PROBLEM_SIZE)

# Create a dictionary to store the fuzzy membership values for each element
fuzzy_values_dict = {elements[i]: membership_values[:, i] for i in range(PROBLEM_SIZE)}

# Implement fuzzy metaheuristics to select sets
population_size = 10
max_iterations = 1000

best_solution = None
best_solution_score = -1

for _ in range(max_iterations):
    candidate_solution = sample(range(NUM_SETS), population_size)
    covered_elements = set()
    score = 0

    for i in candidate_solution:
        covered_elements.update([elements[j] for j in range(PROBLEM_SIZE) if SETS[i][j]])
        score += sum(max(fuzzy_values_dict[elem]) for elem, included in zip(elements, SETS[i]) if included)

    if score > best_solution_score:
        best_solution = candidate_solution
        best_solution_score = score

# Print the results
end = time.time()
print("duration for fuzzy metaheuristics", end - start)
print("Best solution:", best_solution)

duration for fuzzy metaheuristics 6.708080768585205
Best solution: [2, 652, 837, 851, 968, 203, 726, 702, 476, 363]


In [ ]:
s=SETS[542] | SETS[280] | SETS[21] | SETS[824]| SETS[920]| SETS[934]| SETS[312]| SETS[850]| SETS[774]| SETS[245]
print(sum(s))

983
